In [41]:
#Netflows are total count of occurence of a src_ip in a particular window
import pandas as pd
import numpy as np
#Edit path to the file for extracting features
def getFeatures(file_path):
    df=pd.read_csv(file_path)
    temp = pd.get_dummies(df.Tag)
    df['attack'] = temp.Attack
    df['normal'] = temp.Normal
    del temp, df['Tag']
    group = df.groupby(['bucket','source'])
    features = group.sourcePort.nunique()
    features = pd.DataFrame(features)
    features.rename(columns={'sourcePort':'usrc_port'},inplace=True)
    features['udest_ip'] = group.destination.nunique()
    features['udest_port'] = group.destinationPort.nunique()
    features['netflows'] = group.destination.count()
    features['bytes'] = group.totalSourceBytes.sum()
    features['packets'] = group.totalSourcePackets.sum()
    features['attack'] = group.attack.sum()
    features['normal'] = group.normal.sum()
    features.reset_index(inplace=True) 
    return features
    
def featurestoArray(features):
    #Converting DF to np array
    features_array = features.copy()
    del features_array['attack'],features_array['normal'],features_array['source'],features_array['bucket']
    features_array = features_array.as_matrix()
    return features_array

def getClusterFeatures(labeled_features):
    group = labeled_features.groupby(['label'])
    clusterfeatures = group.bucket.count()
    clusterfeatures = pd.DataFrame(clusterfeatures)
    clusterfeatures.rename(columns={'bucket':'instances'},inplace=True) 
    clusterfeatures['netflows']=group.netflows.sum()
    clusterfeatures['avgnetflows']=group.netflows.mean()
    clusterfeatures['stdnetflows']=group.netflows.std()
    clusterfeatures['usrc_ip']=group.source.nunique()
    clusterfeatures['avgsrc_port']=group.usrc_port.mean()
    clusterfeatures['stdsrc_port']=group.usrc_port.std()
    clusterfeatures['avgdest_ip']=group.udest_ip.mean()
    clusterfeatures['stddest_ip']=group.udest_ip.std()
    clusterfeatures['avgdest_port']=group.udest_port.mean()
    clusterfeatures['stddest_port']=group.udest_port.std()
    clusterfeatures['avgbytes']=group.bytes.mean()
    clusterfeatures['stdbytes']=group.bytes.std()
    clusterfeatures['avgpackets']=group.packets.mean()
    clusterfeatures['stdpackets']=group.packets.std()
    clusterfeatures['attack']=group.attack.sum()
    clusterfeatures['normal']=group.normal.sum()
    #True = Botnet    False = Normal
    clusterfeatures['label'] = (group.attack.sum()/group.normal.sum()>0.01)
    return clusterfeatures

In [42]:
def getLabeledFeaturesGMM(features_array):
    X_train = features_array
    from sklearn import mixture
    #Number of clusters = number of components
    # Four covariance Type ['spherical', 'diag', 'tied', 'full'] iterations can be changed
    clf = mixture.GMM(n_components=40, covariance_type='full', n_iter=100)
    label = clf.fit_predict(X_train)
    return label           

def addLabelTofeatures(features, label):
    labeled_features = features.copy()
    labeled_features['label'] = label
    return labeled_features

In [43]:
#features = getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/13Junflow_ndf.csv")
#features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/14junflow_ndf.csv")],axis=0)
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/15junflow1_ndf.csv")],axis=0) 
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/15junflow2_ndf.csv")],axis=0) 
features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/15junflow3_ndf.csv")],axis=0) 
#features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/16junflow1_ndf.csv")],axis=0) 
#features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/16junflow2_ndf.csv")],axis=0) 
#features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/16junflow3_ndf.csv")],axis=0)
#features = pd.concat([features,getFeatures(file_path="/home/ankit/Desktop/MTP/working_directory/ndf/17junflow3_ndf.csv")],axis=0) 


In [44]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer 

X_train, X_test = train_test_split(features, test_size=0.3,random_state=0)
X_train

,bucket,source,usrc_port,udest_ip,udest_port,netflows,bytes,packets,attack,normal
5446,440.0,192.168.3.117,4,5,2,5,3022.0,25.0,0.0,5.0
19175,1312.0,192.168.4.120,1,1,1,1,531.0,8.0,0.0,1.0
15505,556.0,189.12.115.5,1,1,1,1,104.0,1.0,0.0,1.0
22223,708.0,192.168.2.110,15,15,2,16,14139.0,189.0,0.0,16.0
54,54.0,192.168.2.107,13,25,23,25,45933.0,655.0,0.0,25.0
11859,457.0,192.168.2.106,5,5,2,5,2126.0,25.0,0.0,5.0
7622,560.0,192.168.3.117,6,7,2,7,13049.0,119.0,0.0,7.0
1187,65.0,192.168.1.102,9,1,1,9,6068.0,75.0,0.0,9.0
1985,101.0,192.168.5.122,167,9,2,168,24578.0,255.0,0.0,168.0
3018,150.0,192.168.2.110,33,15,2,33,31289.0,419.0,0.0,33.0


In [39]:
#Reading unlabelled test file
#X_test
# df=pd.read_csv("F:/MTP/working_directory/17Jan_df_bucket.csv")
# group = df.groupby(['bucket','SrcAddr'])
# X_test = group.Sport.nunique()
# X_test = pd.DataFrame(X_test)
# X_test['udest_ip'] = group.DstAddr.nunique()
# X_test['udest_port'] = group.Dport.nunique()
# X_test['netflows'] = group.DstAddr.count()
# X_test['bytes'] = group.TotBytes.sum()
# X_test['packets'] = group.sTos.sum()
# X_test['attack'] = group.sTos.sum() #dummy
# X_test['normal'] = group.sTos.sum() #dummy
# X_test.reset_index(inplace=True) 
# X_test.rename(columns={'Sport':'usrc_port'},inplace=True)
# X_test.rename(columns={'SrcAddr':'source'},inplace=True)
# X_test
#df

Converting Dataframe to numpy object

In [45]:
feature_array = featurestoArray(features=X_train)
label = getLabeledFeaturesGMM(features_array=feature_array)
X_train = addLabelTofeatures(features=X_train,label=label)
train_cluster_features = getClusterFeatures(labeled_features=X_train) 
train_labels = train_cluster_features['label']
del train_cluster_features['attack'],train_cluster_features['normal'],train_cluster_features['label']
train_cluster_features = Imputer.fit_transform(Imputer(strategy="most_frequent",axis=0),train_cluster_features)


feature_array = featurestoArray(features=X_test)
label = getLabeledFeaturesGMM(features_array=feature_array)
X_test = addLabelTofeatures(features=X_test,label=label)
test_cluster_features = getClusterFeatures(labeled_features=X_test)
test_labels = test_cluster_features['label']
del test_cluster_features['attack'],test_cluster_features['normal'],test_cluster_features['label'] 
test_cluster_features = Imputer.fit_transform(Imputer(strategy="most_frequent",axis=0),test_cluster_features)
del feature_array

/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/ankit/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:

In [46]:
train_cluster_features = train_cluster_features.astype(np.float32)
test_cluster_features = test_cluster_features.astype(np.float32)
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)
train_cluster_features

array([[  4.51660000e+04,   5.06160000e+04,   1.12066603e+00,
          3.25742304e-01,   2.28900000e+03,   1.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   1.74811340e+02,
          1.77522232e+02,   1.74502945e+00,   2.30692506e+00],
       [  3.00000000e+01,   5.59000000e+03,   1.86333328e+02,
          7.43525314e+01,   1.00000000e+00,   5.05333328e+01,
          1.54579144e+01,   1.71199997e+02,   6.42239609e+01,
          1.49100006e+02,   5.55252953e+01,   6.95144640e+07,
          4.26958281e+04,   4.80176992e+04,   9.11641357e+02],
       [  4.97000000e+02,   1.31600000e+04,   2.64788723e+01,
          4.10930786e+01,   3.60000000e+01,   2.64788723e+01,
          4.10930786e+01,   1.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   0.00000000e+00,   2.27808703e+05,
          3.63415312e+05,   2.18192969e+03,   3.36403198e+03],
       [  2.80000000e+02,   8.32700000e+03,   2.97392864e+01,
     

In [47]:
#normalize cluster features
import sklearn.preprocessing
train_cluster_features= sklearn.preprocessing.normalize(train_cluster_features,axis=0)
test_cluster_features= sklearn.preprocessing.normalize(test_cluster_features,axis=0)

In [48]:
#from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_cluster_features,train_labels)
cluster_label_predict=clf.predict(test_cluster_features)
cluster_label_predict

array([ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
        1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.], dtype=float32)

In [49]:
print sum(abs(cluster_label_predict-test_labels))
temp = X_test.groupby(['label'])
t = temp.packets.count()
label_predicted = [-1 if e == 0 else e for e in cluster_label_predict]
test_labels = [-1 if e == 0 else e for e in test_labels]
#Confusion Matrix of Classification of clusters acccording to decision tree classifier
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels, label_predicted) #0 = normal

3.0


array([[25,  1],
       [ 2, 12]])

In [50]:
#Unique sourse IP's presetn in the test dataset
X_test['pred'] = [label_predicted[x[0]] for x in zip(X_test['label'])]
X_test['act_label'] = [test_labels[x[0]] for x in zip(X_test['label'])]
X_test.source.unique()

array(['192.168.2.113', '192.168.2.106', '192.168.4.118', ...,
       '82.169.169.214', '91.78.103.119', '91.200.45.31'], dtype=object)

In [51]:
#Malicious IP's present in the test dataset
malicious_IP = X_test.groupby(['pred']).get_group(1)['source']
malicious_IP.unique()

array(['192.168.1.103', '192.168.2.106', '192.168.3.116', '192.168.4.118',
       '192.168.1.104', '192.168.1.105', '192.168.2.111', '192.168.3.115',
       '192.168.2.108', '192.168.1.101', '192.168.2.113', '192.168.3.114',
       '192.168.2.107', '192.168.1.102', '192.168.2.109', '192.168.5.122',
       '192.168.4.119', '192.168.4.121', '192.168.2.112', '192.168.3.117',
       '192.168.2.110', '131.202.240.218', '187.60.74.67', '192.168.4.120',
       '131.202.240.209', '142.167.88.44', '67.195.23.155',
       '66.249.67.236', '65.55.34.12', '142.166.115.13', '192.168.5.123',
       '67.195.23.156', '98.139.91.75'], dtype=object)

In [52]:
#Malicious IP's predicted by Model in the test dataset
malicious_act = X_test.groupby(['act_label']).get_group(1)['source']
malicious_act.unique()

array(['192.168.1.103', '192.168.2.106', '192.168.3.116', '192.168.4.118',
       '192.168.1.104', '192.168.1.105', '192.168.3.115', '192.168.2.108',
       '192.168.1.101', '192.168.2.113', '192.168.2.107', '192.168.1.102',
       '192.168.2.109', '192.168.5.122', '192.168.3.114', '192.168.2.111',
       '192.168.4.119', '192.168.2.112', '192.168.3.117', '192.168.2.110',
       '131.202.240.218', '187.60.74.67', '192.168.4.121', '192.168.4.120',
       '131.202.240.209', '67.195.23.155', '66.249.67.236', '65.55.34.12',
       '192.168.5.123', '67.195.23.156', '98.139.91.75'], dtype=object)